<a href="https://colab.research.google.com/github/RosalesLuis123/IA/blob/main/Parcial2/4_en_raya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***4 en raya***

# **Se crea el tablero**

In [ ]:
import numpy as np

class Board():
    def __init__(self):
        self.state = np.zeros((4,4))

    def valid_moves(self):
        return [(i, j) for j in range(4) for i in range(4) if self.state[i, j] == 0]

    def update(self, symbol, row, col):
        if self.state[row, col] == 0:
            self.state[row, col] = symbol
        else:
            raise ValueError("¡Movimiento ilegal!")

    def is_game_over(self):
        # Comprobar filas y columnas
        for i in range(4):
            for j in range(4 - 3):
                if abs(sum(self.state[i, j:j + 4])) == 4:
                    return int(sum(self.state[i, j:j + 4]) / 4)
                if abs(sum(self.state[j:j + 4, i])) == 4:
                    return int(sum(self.state[j:j + 4, i]) / 4)

        # Comprobar diagonales
        for i in range(4 - 3):
            for j in range(4 - 3):
                if abs(sum([self.state[i + k, j + k] for k in range(4)])) == 4:
                    return int(sum([self.state[i + k, j + k] for k in range(4)]) / 4)
                if abs(sum([self.state[i + 3 - k, j + k] for k in range(4)])) == 4:
                    return int(sum([self.state[i + 3 - k, j + k] for k in range(4)]) / 4)

        # Empate
        if len(self.valid_moves()) == 0:
            return 0
        # Seguir jugando
        return None

    def reset(self):
        self.state = np.zeros((4,4))


la clase Game coordina el juego entre dos jugadores, manejando la interacción entre los jugadores y el tablero, así como la asignación de recompensas al final de cada juego.

In [ ]:
class Game():
    def __init__(self, player1, player2):
        player1.symbol = 1
        player2.symbol = -1
        self.players = [player1, player2]
        self.board = Board()

    def selfplay(self, rounds=100):
        wins = [0, 0]
        for i in tqdm(range(1, rounds + 1)):
            self.board.reset()
            for player in self.players:
                player.reset()
            game_over = False
            while not game_over:
                for player in self.players:
                    action = player.move(self.board)
                    self.board.update(player.symbol, action[0], action[1])
                    for player in self.players:
                        player.update(self.board)
                    if self.board.is_game_over() is not None:
                        game_over = True
                        break
            self.reward()
            for ix, player in enumerate(self.players):
                if self.board.is_game_over() == player.symbol:
                    wins[ix] += 1
        return wins

    def reward(self):
        winner = self.board.is_game_over()
        if winner == 0:  # empate
            for player in self.players:
                player.reward(0.5)
        else:  # le damos 1 recompensa al jugador que gana
            for player in self.players:
                if winner == player.symbol:
                    player.reward(1)
                else:
                    player.reward(0)

La clase Agent representa un agente que aprende a jugar mediante un algoritmo de aprendizaje por refuerzo. Utiliza una tabla de valores para estimar la calidad de los diferentes estados del juego y ajusta

In [ ]:
import numpy as np

class Agent():
    def __init__(self, alpha=0.5, prob_exp=0.5):
        self.value_function = {}  # tabla con pares estado -> valor
        self.alpha = alpha        # learning rate
        self.positions = []       # guardamos todas las posiciones de la partida
        self.prob_exp = prob_exp  # probabilidad de explorar
        self.symbol = None        # símbolo del agente en el juego

    def reset(self):
        self.positions = []

    def move(self, board, explore=True):
        valid_moves = board.valid_moves()
        # exploración
        if explore and np.random.uniform(0, 1) < self.prob_exp:
            # vamos a una posición aleatoria
            ix = np.random.choice(len(valid_moves))
            return valid_moves[ix]
        # explotación
        # vamos a la posición con más valor
        max_value = -1000
        for row, col in valid_moves:
            next_board = board.state.copy()
            next_board[row, col] = self.symbol
            next_state = str(next_board.reshape(4*4))  # Ajuste a 4 en raya
            value = 0 if self.value_function.get(next_state) is None else self.value_function.get(next_state)
            if value >= max_value:
                max_value = value
                best_row, best_col = row, col
        return best_row, best_col

    def update(self, board):
        self.positions.append(str(board.state.reshape(4*4)))  # Ajuste a 4 en raya

    def reward(self, reward):
        # al final de la partida (cuando recibimos la recompensa)
        # iteramos por todos los estados actualizando su valor en la tabla
        for p in reversed(self.positions):
            if self.value_function.get(p) is None:
                self.value_function[p] = 0
            self.value_function[p] += self.alpha * (reward - self.value_function[p])
            reward = self.value_function[p]


# **Se entrena el Modelo**

In [ ]:
from tqdm import tqdm  # Importar tqdm


agent1 = Agent(prob_exp=0.5)
agent2 = Agent()

game = Game(agent1, agent2)

game.selfplay(300000)


100%|██████████| 300000/300000 [1:32:06<00:00, 54.29it/s]


[115315, 81204]

Ve las rondas donde d=se gano y perdio 1 gana 0 pierde

In [ ]:
import pandas as pd

funcion_de_valor = sorted(agent1.value_function.items(), key=lambda kv: kv[1], reverse=True)
tabla = pd.DataFrame({'estado': [x[0] for x in funcion_de_valor], 'valor': [x[1] for x in funcion_de_valor]})

tabla

,estado,valor
0,[ 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. -...,1.0
1,[ 0. 0. 0. 1. 0. -1. 0. 1. 0. -1. -1. ...,1.0
2,[ 0. 0. 0. 1. 0. 0. 0. 1. -1. -1. -1. ...,1.0
3,[ 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. -...,1.0
4,[ 0. 0. 0. 1. 0. 0. 0. 1. 0. -1. -1. ...,1.0
...,...,...
1170241,[-1. -1. 0. 0. 1. 1. 0. -1. 0. 1. 1. -...,0.0
1170242,[-1. -1. 0. 0. 1. 1. 0. -1. 0. 1. 1. -...,0.0
1170243,[ 0. -1. 0. 0. 1. 1. 0. -1. 0. 1. 1. -...,0.0
1170244,[-1. 1. 0. -1. -1. 1. 1. 1. 0. 1. 1. -...,0.0


Guardamos los datos de agent

In [ ]:
import pickle

with open('agente.pickle', 'wb') as handle:
    pickle.dump(agent1.value_function, handle, protocol=pickle.HIGHEST_PROTOCOL)